In [1]:
# EEG feature extraction inspired from : https://github.com/Jiaqi1008/Emotion_detection/blob/master/EEG.py
import scipy.io as sio
from scipy import signal
import numpy as np
import pandas as pd
from sklearn import preprocessing as pre
import neurokit2 as nk
import matplotlib.pyplot as plt

All_Features = pd.read_csv("All_Features_with_Age_Gender.csv")
N_epochs=10
fs_EEG = 128 #Hz : EEG sampling rate

def preprocessing(input,feature):
    overall=signal.firwin(9,[0.0625,0.46875],window='hamming')
    theta=signal.firwin(9,[0.0625,0.125],window='hamming')
    alpha=signal.firwin(9,[0.125,0.203125],window='hamming')
    beta=signal.firwin(9,[0.203125,0.46875],window='hamming')
    filtedData=signal.filtfilt(overall,1,input)
    filtedtheta=signal.filtfilt(theta,1,filtedData)
    filtedalpha=signal.filtfilt(alpha,1,filtedData)
    filtedbeta=signal.filtfilt(beta,1,filtedData)
    ftheta,psdtheta=signal.welch(filtedtheta,nperseg=256)
    falpha,psdalpha=signal.welch(filtedalpha,nperseg=256)
    fbeta,psdbeta=signal.welch(filtedbeta,nperseg=256)
    feature.append(max(psdtheta))
    feature.append(max(psdalpha))
    feature.append(max(psdbeta))
    return feature
if __name__ == '__main__':
    total=0
    path=u'DREAMER.mat'
    data=sio.loadmat(path)
    print("EEG signals are being feature extracted...")
    EEG_tmp=np.zeros((23,18,42))
    
EEG_Epochs_Collection = {} 
for e in range(0,N_epochs):
    for k in range(0,23): # Subject
        for j in range(0,18): # Video
            for i in range(0,14): # Electrode    
                basl=data['DREAMER'][0,0]['Data'][0,k]['EEG'][0,0]['baseline'][0,0][j,0][:,i]
                B_len = len(basl)
                stim=data['DREAMER'][0,0]['Data'][0,k]['EEG'][0,0]['stimuli'][0,0][j,0][:,i]
                S_len = len(stim)
                last_index_basl = int(np.floor(B_len/N_epochs)*N_epochs)
                epoch_length_basl = int(np.floor(B_len/N_epochs))
                truncated_basl = np.reshape(np.asarray(basl[0:last_index_basl]),(N_epochs,epoch_length_basl))
                last_index_stim = int(np.floor(S_len/N_epochs)*N_epochs)
                epoch_length_stim = int(np.floor(S_len/N_epochs))
                truncated_stim = np.reshape(np.asarray(stim[0:last_index_stim]),(N_epochs,epoch_length_stim))
                #plt.plot(truncated_basl[1,:])
                #plt.show()
                #for e in range(0,N_epochs):
                B,S=[],[]
                B=preprocessing(truncated_basl[e,:],B)
                S=preprocessing(truncated_stim[e,:],S)
                Extrod=np.divide(S,B)
                total+=1
                EEG_tmp[k,j,3*i]=Extrod[0]
                EEG_tmp[k,j,3*i+1]=Extrod[1]
                EEG_tmp[k,j,3*i+2]=Extrod[2]
                print("\rprogress: %d%%" %(total/(23*18*14*N_epochs)*100),end="")
                #print("Shape : "+str(np.shape(EEG_tmp)))
    col=[]
    for i in range(0,14):
        #for e in range(0,N_epochs):
            col.append('psdtheta_'+str(i + 1)+'_un')
            col.append('psdalpha_'+str(i + 1)+'_un')
            col.append('psdbeta_'+str(i + 1)+'_un')
    #print(np.shape(col))
    EEG=pd.DataFrame(EEG_tmp.reshape((23 * 18,EEG_tmp.shape[2])),columns=col)
    #EEG=pd.DataFrame(EEG_tmp.reshape((23 * 18,EEG_tmp.shape[2]*EEG_tmp.shape[3])),columns=col)
    scaler=pre.StandardScaler()
    for i in range(len(col)):
        #for e in range(0,N_epochs):
            #print(np.shape(col))
            EEG[col[i][:-3]]=scaler.fit_transform(EEG[[col[i]]])
    EEG.drop(col,axis=1,inplace=True)
    EEG_Epochs_Collection[e]=EEG
    EEG_Epochs_Collection[e] = EEG_Epochs_Collection[e].join(All_Features['Valence'])
    EEG_Epochs_Collection[e] = EEG_Epochs_Collection[e].join(All_Features['Arousal'])
    #print(EEG)
    #EEG.to_csv(str('Extracted_EEG_'+str(N_epochs)+'_epochs.csv'))

EEG signals are being feature extracted...
progress: 100%

In [5]:
# min-max scaling
for e in range(0,N_epochs):
    for column in EEG_Epochs_Collection[e].columns:
        if not(EEG_Epochs_Collection[e][column].dtype == np.object):
            if not((np.max(EEG_Epochs_Collection[e][column])-np.min(EEG_Epochs_Collection[e][column]))==0):
                EEG_Epochs_Collection[e][column]=(EEG_Epochs_Collection[e][column]-np.min(EEG_Epochs_Collection[e][column]))/(np.max(EEG_Epochs_Collection[e][column])-np.min(EEG_Epochs_Collection[e][column]))


In [6]:
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.express as px

Epoch_IntSlider = widgets.IntSlider(
    value=1,
    min=1,
    max=N_epochs,
    step=1,
    description='Epoch:',
    continuous_update=False
)

e = Epoch_IntSlider.value-1
corrMatrix = EEG_Epochs_Collection[e].corr()
g = go.FigureWidget(data=go.Heatmap(z=corrMatrix,y=EEG_Epochs_Collection[e].columns,x=EEG_Epochs_Collection[e].columns),
                    layout=go.Layout(
                        title=dict(
                            text='Cross-correlations for different epochs (EEG)'
                        ),
                        barmode='overlay'
                    ))

"""heatmap_corrMatrix = px.imshow(corrMatrix,
                               title='(z-scored)(Valence and Arousal Thresholded) Cross-correlation matrix (Ratio of All_Features_with_Age_Gender.csv/Extracted_EEG_last4s.csv)',
                               
                y=EEG_Epochs_Collection[e].columns,
                x=EEG_Epochs_Collection[e].columns#[str('f'+str(j+1)) for j in range(0,Nf-4)]
                )
heatmap_corrMatrix.show()
"""
def validate():
    if Epoch_IntSlider.value in range(1,N_epochs+1):
        return True
    else:
        return False
def response(change):
    if validate():
        #e = Epoch_IntSlider.value -1
        #if use_date.value:
        #    filter_list = [i and j and k for i, j, k in
        #                   zip(df['month'] == month.value, df['carrier'] == textbox.value,
        #                       df['origin'] == origin.value)]
        #    temp_df = df[filter_list]

        #else:
        #    filter_list = [i and j for i, j in
        #                   zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
        #    temp_df = df[filter_list]
        #x1 = temp_df['arr_delay']
        #x2 = temp_df['dep_delay']
        with g.batch_update():
            e = Epoch_IntSlider.value-1
            g.data[0].z = EEG_Epochs_Collection[e].corr()
            g.layout.barmode = 'overlay'
Epoch_IntSlider.observe(response, names="value")
container = widgets.HBox(children=[Epoch_IntSlider])



widgets.VBox([container,g])